In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

## Download Data

In [40]:
import os

cwd = os.getcwd()
# Check if price data have been saved
price_data = os.path.join(cwd, "price_data")
if not os.path.exists(price_data):
    # Getting the Nasdaq 100 ticker symbols from Wikipedia
    nas_df = pd.read_html("https://en.wikipedia.org/wiki/Nasdaq-100")[4]
    tickers = nas_df.Ticker.to_list()
    # Download the Nasdaq 100 Adjusted Close price data with yfinance
    df = yf.download(tickers, start="2010-01-01")
    # Create price_data directory
    os.makedirs(price_data)
    # Save a copy of the dataframe to price_data directory
    df.to_csv(os.path.join(price_data, "Nasdaq-100.csv"))
else:
    # Load the Nasdaq-100.csv
    df = pd.read_csv(os.path.join(price_data, "Nasdaq-100.csv"), 
                     header=[0,1], index_col=0)
    df.index = pd.to_datetime(df.index)

In [45]:
df = df.swaplevel(axis=1)

## Percent Change Calculation

In [21]:
def batching(data, bs=2):
    length = len(data)
    n_batch = length // bs
    max_len = n_batch * bs
    return np.array(data)[:max_len].reshape(n_batch, bs)

def regrouped(data, bs=2):
    date = batching(data.index, bs=bs)[:, -1]
    close = batching(data['Close'], bs=bs)
    high = batching(data['High'], bs=bs)[:, -1:].max(1)
    base = close[:, -1] / close[:, 0]
    return pd.Series(base, index=date)

In [46]:
tickers = df.columns.levels[0]

In [55]:
records = None
freqs = ['D', 'W', 'M', 'Q']
archives = {}

for ticker in tickers:
    rec = {}
    for freq in freqs:
        if freq == 'D':
            ret = regrouped(df[ticker])
        else:
            wf = df[ticker].loc[:, 'Close'].groupby(by=pd.Grouper(freq=freq)).first()
            wl = df[ticker].loc[:, 'Close'].groupby(by=pd.Grouper(freq=freq)).last()
            ret = wl / wf
        ret = ret.dropna()
        rec[freq] = {'std': ret.std(),
                     'prod': ret.prod(),
                     'avg_gain': np.mean(ret[ret > 1])-1,
                     'avg_loss': np.mean(ret[ret < 1])-1,
                     'max_gain': np.max(ret)-1,
                     'max_loss': np.min(ret)-1, 
                     'win_rate': np.mean(ret > 1)}
    rec = pd.DataFrame.from_dict(rec, orient='index')
    name = [[ticker], rec.columns]
    col = pd.MultiIndex.from_product(name)
    rec.columns = col
    if records is None:
        records = rec
    else:
        records = pd.concat([records, rec], axis=1)


In [61]:
records

AAPL                                                               \
        std       prod  avg_gain  avg_loss  max_gain  max_loss  win_rate   
D  0.017517   7.367225  0.012933 -0.011899  0.119808 -0.123558  0.533862   
W  0.034671   2.169956  0.027528 -0.024734  0.120768 -0.128554  0.505882   
M  0.078969   8.805414  0.067888 -0.062924  0.219542 -0.196382  0.611465   
Q  0.155456  15.099922  0.142858 -0.103730  0.514258 -0.305905  0.679245   

       ABNB                      ...        ZM                            ZS  \
        std      prod  avg_gain  ...  max_gain  max_loss  win_rate       std   
D  0.035191  0.583977  0.027737  ...  0.222214 -0.166906  0.491489  0.039124   
W  0.064986  1.760109  0.049746  ...  0.240419 -0.178431  0.494898  0.078043   
M  0.128453  0.682298  0.099640  ...  0.295323 -0.242095  0.456522  0.146471   
Q  0.228580  0.626363  0.144476  ...  1.126310 -0.363779  0.375000  0.321341   

                                                               
       prod  avg_gain  avg_loss  max_gain  max_loss  win_rate  
D  2.225059  0.028309 -0.025685  0.264459 -0.154545  0.512356  
W  6.179774  0.060021 -0.053092  0.387539 -0.234441  0.557312  
M  1.968787  0.123433 -0.107122  0.467315 -0.280341  0.559322  
Q  3.020206  0.309930 -0.182516  0.781713 -0.392754  0.571429  

[4 rows x 707 columns]